# 模块导入

In [1]:
from sklearn.preprocessing import LabelEncoder
import pandas as pd
import pymysql
import datetime
import warnings
import time
import re
import numpy as np
warnings.filterwarnings('ignore')

# 连接数据库

In [2]:
conn = pymysql.connect(host = 'am-wz9el267w54i2r7ip131930o.ads.aliyuncs.com' # 连接名称，默认127.0.0.1
,user = 'yjy_user' # 用户名
,passwd='Yjy123456' # 密码
,port= 3306 # 端口，默认为3306
,db='yun-campusjob'#数据库名称
,charset='utf8' # 字符编码
)

# 定义函数和参数

In [3]:
percent = 10#前10%平均值作为最大值
lowest_score = 0
# 获取最大值
def get_max_score(s,score_name = 'score'):
    s_temp = s[s[score_name]>0].sort_values(by=score_name,ascending = False)[score_name]
    percent_num = s_temp.shape[0]*percent//100+1
    return int(s_temp.iloc[:percent_num].mean(axis=0))
# 分数转换函数
def score_change(s,max_score,score_name = 'score'):
    temp = s[score_name].apply(lambda x: x*100/max_score if x*100/max_score<100 else 100)
    return temp

# 学生干部

In [4]:
sql = f"SELECT student_key,position,department_name,start_date,duration FROM sch_student_grow_up_leadership_position WHERE school_id = 2426"
data_duties = pd.read_sql(con=conn,sql=sql)

data_duties['校级'] = 1
data_duties.loc[data_duties.query("department_name.str.contains('校')",engine='python').index,'校级']=2#校级干部

data_duties.loc[data_duties.query("duration.str.contains('两年|二年')",engine='python').index,'time_type'] = '两年及以上'
data_duties.loc[data_duties.query("time_type.isnull() and duration.str.contains('年') and duration.str.contains('[0-9]')",engine='python').index,'time_type'] = data_duties.query("time_type.isnull() and duration.str.contains('年') and duration.str.contains('[0-9]')",engine='python')['duration'].apply(lambda x: '两年及以上' if int(re.sub('学年|年','',x))>=2 else np.nan)
data_duties.loc[data_duties.query("time_type.isnull() and duration.str.contains('月') and duration.str.contains('[0-9]')",engine='python').index,'time_type'] = data_duties.query("time_type.isnull() and duration.str.contains('月') and duration.str.contains('[0-9]')",engine='python')['duration'].apply(lambda x: '两年及以上' if int(re.sub('月|个月','',x))>=24 else np.nan)
data_duties.loc[data_duties.query("time_type.isnull() and duration.str.contains('一年|一学年|12')",engine='python').index,'time_type'] = '一至两年'
data_duties.loc[data_duties.query("time_type.isnull() and duration.str.contains('年') and duration.str.contains('[0-9]')",engine='python').index,'time_type'] = data_duties.query("time_type.isnull() and duration.str.contains('年') and duration.str.contains('[0-9]')",engine='python')['duration'].apply(lambda x: '一至两年' if int(re.sub('学年|年','',x))==1 else np.nan)
data_duties.loc[data_duties.query("time_type.isnull() and duration.str.contains('月') and duration.str.contains('[0-9]')",engine='python').index,'time_type'] = data_duties.query("time_type.isnull() and duration.str.contains('月') and duration.str.contains('[0-9]')",engine='python')['duration'].apply(lambda x: '一至两年' if int(re.sub('月|个月','',x))>=12 and int(re.sub('月|个月','',x))<24 else np.nan)
data_duties.loc[data_duties.query("time_type.isnull() and duration.str.contains('至今')",engine='python').index,'time_type'] = data_duties.query("time_type.isnull() and duration.str.contains('至今')",engine='python')['start_date'].apply(lambda x: '两年及以上' if round(abs((pd.to_datetime(x)-datetime.datetime.now()).days)/365,2)>=2 else ('一至两年' if round(abs((pd.to_datetime(x)-datetime.datetime.now()).days)/365,2)>=1 else '一年以下'))
data_duties.loc[data_duties.query("time_type.isnull()").index,'time_type'] = '无'

encoder = LabelEncoder()
encoder.fit(["无", "一年以下", "一至两年", "两年及以上"])
data_duties['time_type']=encoder.transform(data_duties['time_type'])

data_duties['score']=data_duties['校级']*data_duties['time_type']
data_duties = data_duties[['student_key','score']]
data_duties = data_duties.groupby('student_key').sum()
max_score = get_max_score(data_duties)
data_duties['score'] = score_change(data_duties,max_score)
data_duties.columns = ['duties_score']
data_duties

,duties_score
student_key,
0015f4a34537b5a728ef209cf9666ae5,25.0
0035b7565d61cfadccca883b67cbdfea,12.5
0036e7fe0aeb8b14bd2cabb27c3807b4,25.0
0054dc0f5026761fe7588a4be417af78,12.5
005e7445871e729981a1fd4b96ed72a2,25.0
...,...
ff3d9cd305a3a6e29721d59c757471cb,12.5
ff54a86e23710751f2eb02635c47ba22,12.5
ff9f7e0562ee6ac9e73c18694de753e7,25.0


# 惩罚记录数据

In [5]:
sql = f"SELECT student_key,penalty_level,cfqx FROM sch_student_grow_up_penalty WHERE school_id = 2426"
data_penalty = pd.read_sql(con=conn,sql=sql)
value = data_penalty['penalty_level'].astype('category').cat.set_categories(['警告','严重警告','记过','留校察看'])
data_penalty['score'] = value.cat.codes + 1
data_penalty =data_penalty[['student_key','score']].groupby('student_key').sum()
max_score = 4
data_penalty['score'] = score_change(data_penalty,max_score)
data_penalty.columns = ['penalty_score']
data_penalty

,penalty_score
student_key,
00800b4bdc9802457d7f90d393a62b91,25.0
00e82c6da4834a8fb9e33050444a1b70,25.0
00ec285c309d3b8efa6ade79d2ed290f,25.0
021daf99aa95875f62d3614da82c85ad,25.0
02292ffd89f985860cc4531a342bbf14,75.0
...,...
feb9b54662312e31592513bcd50d27c4,25.0
ff5382763ff4228f5782b681c7c92302,25.0
ff75560de677e07a49757b2946d28fa6,25.0


# 实习实践数据

In [6]:
sql = f"SELECT student_key,duration FROM sch_student_grow_up_practice WHERE school_id = 2426"
data_social = pd.read_sql(con=conn,sql=sql)
data_social['score'] = data_social['duration'].apply(lambda x:int(re.findall("\d+",x)[0])*100//max_score)
data_social = data_social[['student_key','score']]
data_social=data_social.groupby('student_key').sum()
max_score = 168
data_social['score'] = score_change(data_social,max_score)
data_social.columns = ['shixi_score']
data_social

,shixi_score
student_key,
0000aa46d5dcb6d3cb69e2061a9d24b4,100.000000
0000b10bcd2240a7490230d08d77a68b,100.000000
0015f4a34537b5a728ef209cf9666ae5,100.000000
001d6150c54d38f2ef047826a1e02f83,100.000000
002ee528b425df261ac5cf5cb90680d9,100.000000
...,...
ffd40477b1dc2f769c7c1a36b659d3dc,100.000000
ffde64d74eb85514ad3e65c3b2315ff3,44.642857
fff207e47231e3ed7f2f565b158cb9de,100.000000


# 科研数据

In [7]:
sql = f"SELECT student_key,grade,college_evaluate,school_evaluate FROM sch_student_grow_up_research WHERE school_id = 2426"
data_tech = pd.read_sql(con=conn,sql=sql)
value_1 = data_tech['grade'].astype('category').cat.set_categories(['校级','校重点','省级','国家级'])
data_tech['v1'] = value_1.cat.codes + 1
value_2 = data_tech['college_evaluate'].astype('category').cat.set_categories(['不合格','合格','良好','优秀'])
data_tech['v2'] = value_2.cat.codes + 1
value_3 = data_tech['school_evaluate'].astype('category').cat.set_categories(['不合格','合格','良好','优秀'])
data_tech['v3'] = value_3.cat.codes + 1
data_tech['score'] = data_tech['v1']*data_tech['v2']*data_tech['v3']
data_tech = data_tech[['student_key','score']]
data_tech=data_tech.groupby('student_key').sum()
max_score = get_max_score(data_tech)
data_tech['score'] = score_change(data_tech,max_score)
data_tech.columns = ['tech_score']
data_tech

,tech_score
student_key,
00012980d6169ab36bf349ce65bca0e0,75.0000
0004d13c843b3ae8a7469856f9942d18,14.0625
000a80bc93038c9644c608ea1d0452ec,18.7500
001b3c85fcd8978c4ccce6c80e63a6a7,81.2500
003129629aac4ec3925036cf5050bfee,25.0000
...,...
ff8f93b63c9d0dbac5721320f13b3b7f,75.0000
ffb3d455fd08a684609a569ab0c8325c,25.0000
ffc13f054605d26ccfb4b4c545205648,14.0625


# 招聘会数据

In [8]:
sql = f"SELECT student_key,count(*) as score FROM sch_student_grow_up_visit_career_fair WHERE school_id = 2426 GROUP BY student_key"
data_fair = pd.read_sql(con=conn,sql=sql)
max_score = get_max_score(data_fair)
data_fair['score'] = score_change(data_fair,max_score)
data_fair.columns = ['student_key','fair_score']
data_fair

,student_key,fair_score
0,34fc8a605ef11b2fb34ff314d9b091ab,25.0
1,e763b81ffd1dedb5854790b92f6cadbb,25.0
2,671a4becdc0cf2065bb507e7316a846b,25.0
3,68b75a418bda19d2c5375f1e010bc7ac,25.0
4,59f1832679593c4243d5198cf37e9c6b,75.0
...,...,...
6404,2978085981bd434720703e61a0df9c2a,25.0
6405,43cd22c3ed9833628bac93ebfeabcec2,25.0
6406,634c32ca1334f3a3fad97f9816b0acf8,25.0
6407,84caf9ba04ecbf0e711bd5f5c285d70f,25.0


# 社会实践数据

In [9]:
sql = f"SELECT student_key,sum(num) as score FROM sch_student_grow_up_social_practice WHERE school_id = 2426 GROUP BY student_key"
data_sci_p = pd.read_sql(con=conn,sql=sql)
max_score = get_max_score(data_sci_p)
data_sci_p['score'] = score_change(data_sci_p,max_score)
data_sci_p.columns = ['student_key','sci_score']
data_sci_p

,student_key,sci_score
0,85d82a6f21e6d5364e18223d606d5adc,100.0
1,6342cf0a9e3a080f7844004655249bfa,90.0
2,4fd168f6f4531fd18fc1f0de3a7092b0,25.0
3,ec35acc5e7472d242e5222d0d3b6c8cc,20.0
4,306b1615f6bf21df6c9b3f9aa35bb3d8,20.0
...,...,...
6301,e411b432eef4238a71d85f2dce5a94f7,20.0
6302,cbf22248a3284706d750315a17cc65bb,15.0
6303,46ce26a0ea2b0799286c8413d84deda8,10.0
6304,4dc754b91925f7e009f29f36695ceab7,10.0


# 熵值法

In [10]:
def cal_weight(column,row,value):
    p= np.array([[0.0 for i in range(column)] for i in range(row)])     
    for i in range(column):
        p[:,i]=value[:,i]/np.sum(value[:,i],axis=0)
    e=-1/np.log(row)*sum(p*np.log(p+1e-10))      #计算熵值
    g=1-e     # 计算一致性程度
    w=g/sum(g)     #计算权重
    return w

# 四六级，竞赛，奖学金，自主添加项权重计算

In [11]:
sql = f"SELECT student_key,honor_name,dept FROM sch_student_grow_up_honor WHERE school_id = 2426"
data_honor = pd.read_sql(con=conn,sql=sql)
data_honor['level'] = 1

data_honor.loc[data_honor.query("honor_name.str.contains('四六级（补录）|英语四六级考试')",engine='python').index,'honor_name']='四六级'
data_honor.loc[data_honor.query("honor_name.str.contains('数学竞赛')",engine='python').index,'level']=2
data_honor.loc[data_honor.query("honor_name.str.contains('竞赛')",engine='python').index,'honor_name']='学科竞赛'
data_honor.loc[data_honor.query("honor_name.str.contains('国家奖学金')",engine='python').index,'level']=5
data_honor.loc[data_honor.query("honor_name.str.contains('校长奖学金')",engine='python').index,'level']=4
data_honor.loc[data_honor.query("honor_name.str.contains('专业一等奖学金|一等专业奖学金')",engine='python').index,'level']=3
data_honor.loc[data_honor.query("honor_name.str.contains('二等专业奖学金')",engine='python').index,'level']=2
data_honor.loc[data_honor.query("honor_name.str.contains('奖学金')",engine='python').index,'honor_name']='奖学金'
data_honor.loc[data_honor[data_honor['honor_name'].isin(['四六级','学科竞赛','奖学金'])==False].index,'honor_name']='其他'

data_honor['46_score'] = data_honor.apply(lambda x:1 if x['honor_name'] == '四六级' else 0,axis=1)
data_honor['jingsai_score'] = data_honor.apply(lambda x:x['level'] if x['honor_name'] == '学科竞赛' else 0,axis=1)
data_honor['money_score'] = data_honor.apply(lambda x:x['level'] if x['honor_name'] == '奖学金' else 0,axis=1)
data_honor['other_score'] = data_honor.apply(lambda x:x['level'] if x['honor_name'] == '其他' else 0,axis=1)

data_study_ability = data_honor[['student_key','jingsai_score','money_score','other_score','46_score']]
data_study_ability = data_study_ability.groupby('student_key').sum()

max_score = get_max_score(data_study_ability,'jingsai_score')
data_study_ability['jingsai_score'] = score_change(data_study_ability,max_score,'jingsai_score')
max_score = get_max_score(data_study_ability,'other_score')
data_study_ability['other_score'] = score_change(data_study_ability,max_score,'other_score')
max_score = get_max_score(data_study_ability,'46_score')
data_study_ability['46_score'] = score_change(data_study_ability,max_score,'46_score')
max_score = get_max_score(data_study_ability,'money_score')
data_study_ability['money_score'] = score_change(data_study_ability,max_score,'money_score')
data_study_ability

,jingsai_score,money_score,other_score,46_score
student_key,,,,
0007e830c0d985e81e1f2643bba16438,66.666667,0.0,0.000000,0.0
000c0400f996e8df3f59230cb8f5b7f1,66.666667,0.0,0.000000,0.0
001d6150c54d38f2ef047826a1e02f83,0.000000,0.0,33.333333,0.0
002fb2a22f19ee062bb23853f907084a,33.333333,0.0,0.000000,0.0
0030c6de1837fd26a22d6ca06cb3cb2d,0.000000,0.0,33.333333,0.0
...,...,...,...,...
ffde64d74eb85514ad3e65c3b2315ff3,0.000000,0.0,33.333333,0.0
fff207e47231e3ed7f2f565b158cb9de,33.333333,0.0,33.333333,0.0
fff6c463b8d19a7b588a293def7be299,33.333333,0.0,33.333333,0.0


In [12]:
sql = f"SELECT distinct student_key FROM sch_school_student WHERE school_id = 2426"
data_stu = pd.read_sql(con=conn,sql=sql)
data_study_ability = pd.merge(data_study_ability,data_stu,how='right',on = 'student_key').fillna(0)
w = cal_weight(data_study_ability.shape[1]-1,data_study_ability.shape[0],data_study_ability[['46_score','jingsai_score','money_score','other_score']].values)
data_study_ability['score']=data_study_ability.apply(lambda x:x['46_score']*w[0]+x['jingsai_score']*w[1]+x['money_score']*w[2]+x['other_score']*w[3],axis=1)
w

array([0.2545834 , 0.16215966, 0.43235735, 0.15089958])

# 就业权重计算

In [13]:
data_sum = data_stu
data_list = [data_duties, data_penalty, data_social, data_fair, data_sci_p]
for i in range(len(data_list)):
    data_sum = pd.merge(data_sum, data_list[i], how='left', on='student_key').fillna(0)

data_list = ['duties_score','penalty_score','shixi_score','fair_score','sci_score']
data = data_sum[data_list]
w1 = cal_weight(data.shape[1],data.shape[0],data.values)

#意向准备加分项——就业部分的分数
data_sum['jiuye_score']=0
for i in range(len(data_list)):
    if data_list[i]!='penalty_score':
        data_sum['jiuye_score'] = w1[i]*data_sum[data_list[i]]
    else:
        data_sum['jiuye_score'] = w1[i]*(100-data_sum[data_list[i]])
data_sum
w1

array([0.21339566, 0.27424411, 0.16721892, 0.16790322, 0.1772381 ])